In [ ]:
from rnn.model import *
from rnn.train import *
from rnn.predict import *
from utils.data_loader import *

In [ ]:
# Hyper params
batch_size = 1
sequence_len = 200
# Data path
rawdata_dir = 'raw_data'
middata_dir = 'mid_data'
poem_dir = "poem_result"
model_dir = "rnn_models"

In [ ]:
# loda data with preprocessing
loader = data_loader(sequence_len, batch_size, rawdata_dir, middata_dir)
loader.file_preprocess()

In [ ]:
# load data from file
# data loader
loader = data_loader(sequence_len, batch_size, rawdata_dir, middata_dir)
loader.load()

In [ ]:
# load functionality check
cb = loader.cb
data_batch = loader.np_batchs
print("Input Vector Size : " + str(cb.size()))
print("Total Batch Num : " + str(len(data_batch)))
print(data_batch)

In [ ]:
# Training New Model
# Model Params
input_size = cb.size()
hidden_size = 100
output_size = cb.size()
n_layers = 1

# Model
decoder = RNN(input_size, hidden_size, output_size, n_layers)
optimizer = torch.optim.Adam(decoder.parameters(), lr)
criterion = nn.CrossEntropyLoss()

In [ ]:
# Training Exist Model
model_name = "rnn_model_result.pt"
decoder = torch.load(model_dir + "/" + model_name)
optimizer = torch.optim.Adam(decoder.parameters(), lr)
criterion = nn.CrossEntropyLoss()

In [ ]:
# Training Script
# Hyper Params
lr = 0.003
n_epochs = 2000
print_cycle = 100
save_cycle = 10000
start_sequence = cb.get_number_batch("apple")


start = time.time()

try:
    if (n_epochs == -1):
        epoch = 0
        while (True):
            epoch += 1
            loss = train(decoder, optimizer, criterion, sequence_len, batch_size , *random_training_set(data_batch, sequence_len, batch_size))
            
            if epoch % print_cycle == 0:
                print('[Time : %s, Epoach : (%d), Loss : %.4f]' % (time_since(start), epoch, loss))
                print(cb.get_string(predict(decoder, start_sequence, 100)))
            if (epoch % save_cycle == 0):
                model_num += 1
                print("Save %d model" % (model_num))
                f = open("./" + poem_dir + "/poem" + str(model_num) + ".txt", 'w')
                predicted = predict(decoder, start_sequence, 1000)
                f.write(cb.get_string(predicted))
                save(decoder, "./"+model_dir+"/rnn_model"+str(model_num))
                f.close()

    else :
        print("Training for %d epochs..." % n_epochs)
        for epoch in range(1, n_epochs + 1):
            loss = train(decoder, optimizer, criterion, sequence_len, batch_size , *random_training_set(data_batch, sequence_len, batch_size))

            if epoch % print_cycle == 0:
                print('[Time : %s, Epoach : (%d), Loss : %.4f]' % (time_since(start), epoch, loss))
                print(cb.get_string(predict(decoder, start_sequence, 100)))

        print("Learning end.. Saving...")
        f = open("./" + poem_dir + "/poem_result.txt", 'w')
        predicted = predict(decoder, start_sequence, 1000)
        f.write(cb.get_string(predicted))
        save(decoder, "./"+model_dir+"/rnn_model_result")
        f.close()
        
except KeyboardInterrupt:
    print("Saving before quit...")
    save(decoder, "./"+model_dir+"/rnn_model_result")
    
except :
    print("Learning Failure!! Saving before quit...")
    save(decoder, "./"+model_dir+"/rnn_model_result")